# Module 3. Deployment on MMS(Multi Model Server)
---

본 모듈에서는 모델의 배포(deployment)를 수행합니다. 노트북 실행에는 약 15분 가량 소요되며, 핸즈온 실습 시에는 25분을 권장드립니다.

<br>

## 1. Inference script
---

아래 코드 셀은 `src` 디렉토리에 SageMaker 추론 스크립트인 `inference.py`를 저장합니다.<br>

이 스크립트는 SageMaker 상에서 MMS(Multi Model Server)를 쉽고 편하게 배포할 수 이는 high-level 툴킷인 SageMaker inference toolkit의 인터페이스를
사용하고 있으며, 여러분께서는 인터페이스에 정의된 핸들러(handler) 함수들만 구현하시면 됩니다.

#### MMS(Multi Model Server)란?
- [https://github.com/awslabs/multi-model-server](https://github.com/awslabs/multi-model-server) (2017년 12월 초 MXNet 1.0 릴리스 시 최초 공개, MXNet용 모델 서버로 시작)
- Prerequisites: Java 8, MXNet (단, MXNet 사용 시에만)
- MMS는 프레임워크에 구애받지 않도록 설계되었기 때문에, 모든 프레임워크의 백엔드 엔진 역할을 할 수 있는 충분한 유연성을 제공합니다.
- SageMaker MXNet 추론 컨테이너와 PyTorch 추론 컨테이너는 SageMaker inference toolkit으로 MMS를 래핑하여 사용합니다.
    - 2020년 4월 말 PyTorch용 배포 웹 서비스인 torchserve가 출시되면서, 향후 PyTorch 추론 컨테이너는 MMS 기반에서 torchserve 기반으로 마이그레이션될 예정입니다. 

In [1]:
%%writefile ./src/inference.py

import os
import pandas as pd
import gluonts 
import numpy as np
import argparse
import json
import pathlib
from mxnet import gpu, cpu
from mxnet.context import num_gpus
import matplotlib.pyplot as plt

from gluonts.dataset.util import to_pandas
from gluonts.distribution import DistributionOutput, StudentTOutput, NegativeBinomialOutput, GaussianOutput
from gluonts.model.deepar import DeepAREstimator
from gluonts.trainer import Trainer
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions, backtest_metrics
from gluonts.model.predictor import Predictor
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.common import ListDataset


def model_fn(model_dir):
    path = pathlib.Path(model_dir)   
    predictor = Predictor.deserialize(path)
    print("model was loaded successfully")
    return predictor


def transform_fn(model, request_body, content_type='application/json', accept_type='application/json'):

    data = json.loads(request_body)
    target_test_df = pd.DataFrame(data['value'], index=data['timestamp'])
    target = target_test_df.values
    num_series = target_test_df.shape[1]
    start_dt = target_test_df.index[0]
    test_lst = []
    
    for i in range(0, num_series):
        target_vec = target[:, i]
        dic = {FieldName.TARGET: target_vec, 
               FieldName.START: start_dt} 
        test_lst.append(dic)
        
    test_ds = ListDataset(test_lst, freq='1D')
    
    response_body = {}
    forecast_it = model.predict(test_ds)
    for idx, f in enumerate(forecast_it):
        response_body[f'item_{idx}'] = f.samples.mean(axis=0).tolist()

    return json.dumps(response_body)

Overwriting ./src/inference.py


<br>

## 2. Test Inference code 
---

엔드포인트 배포 전, 추론 스크립트를 검증합니다. 

In [2]:
%store -r

In [3]:
from src.inference import model_fn, transform_fn
import json
import numpy as np
import pandas as pd

# Prepare test data
target_test_df = pd.read_csv("data/target_train.csv")
target_test_df.set_index(target_test_df.columns[0], inplace=True)
num_series = 2
pred_length = 21
target_test_df = target_test_df.iloc[pred_length:,:num_series]
input_data = {'value': target_test_df.values.tolist(), 'timestamp': target_test_df.index.tolist()}
request_body = json.dumps(input_data)

# Test inference script 
model = model_fn('./model')
response = transform_fn(model, request_body)
outputs = json.loads(response)
print(outputs['item_0'])

model was loaded successfully
[13.770057678222656, 16.68670654296875, 17.044038772583008, 18.174997329711914, 20.137741088867188, 20.506511688232422, 21.528583526611328, 14.461799621582031, 16.67385482788086, 16.653356552124023, 17.966821670532227, 19.51350212097168, 20.650901794433594, 21.450462341308594, 14.042078018188477, 16.574562072753906, 17.086204528808594, 17.647796630859375, 19.783123016357422, 19.981788635253906, 20.694074630737305]


<br>

## 3. Local Endpoint Inference
---

충분한 검증 및 테스트 없이 훈련된 모델을 곧바로 실제 운영 환경에 배포하기에는 많은 위험 요소들이 있습니다. 따라서, 로컬 모드를 사용하여 실제 운영 환경에 배포하기 위한 추론 인스턴스를 시작하기 전에 노트북 인스턴스의 로컬 환경에서 모델을 배포하는 것을 권장합니다. 이를 로컬 모드 엔드포인트(Local Mode Endpoint)라고 합니다.

In [4]:
import os
import time
import sagemaker
from sagemaker.mxnet import MXNetModel
role = sagemaker.get_execution_role()

In [5]:
local_model_path = f'file://{os.getcwd()}/model/model.tar.gz'
endpoint_name = "local-endpoint-store-item-demand-forecast-{}".format(int(time.time()))


아래 코드 셀을 실행 후, 로그를 확인해 보세요. MMS에 대한 세팅값들을 확인하실 수 있습니다.

```bash
algo-1-u3xwd_1  | MMS Home: /usr/local/lib/python3.6/site-packages
algo-1-u3xwd_1  | Current directory: /
algo-1-u3xwd_1  | Temp directory: /home/model-server/tmp
algo-1-u3xwd_1  | Number of GPUs: 0
algo-1-u3xwd_1  | Number of CPUs: 2
algo-1-u3xwd_1  | Max heap size: 878 M
algo-1-u3xwd_1  | Python executable: /usr/local/bin/python3.6
algo-1-u3xwd_1  | Config file: /etc/sagemaker-mms.properties
algo-1-u3xwd_1  | Inference address: http://0.0.0.0:8080
algo-1-u3xwd_1  | Management address: http://0.0.0.0:8080
algo-1-u3xwd_1  | Model Store: /.sagemaker/mms/models
...
```

In [6]:
local_model = MXNetModel(model_data=local_model_path,
                         role=role,
                         source_dir='src',
                         entry_point='inference.py',
                         framework_version='1.6.0',
                         py_version='py3')

predictor = local_model.deploy(instance_type='local', 
                           initial_instance_count=1, 
                           endpoint_name=endpoint_name,
                           wait=True)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


Attaching to tmp65wkqgnh_algo-1-su6vt_1
algo-1-su6vt_1  | Collecting pandas==1.0.0
algo-1-su6vt_1  |   Downloading pandas-1.0.0-cp36-cp36m-manylinux1_x86_64.whl (10.1 MB)
     |████████████████████████████████| 10.1 MB 16.7 MB/s eta 0:00:01
algo-1-su6vt_1  | Collecting gluonts==0.5.1
algo-1-su6vt_1  |   Downloading gluonts-0.5.1-py3-none-any.whl (419 kB)
     |████████████████████████████████| 419 kB 43.7 MB/s eta 0:00:01
algo-1-su6vt_1  | Requirement already satisfied: python-dateutil>=2.6.1 in /usr/local/lib/python3.6/site-packages (from pandas==1.0.0->-r /opt/ml/model/code/requirements.txt (line 1)) (2.8.1)
algo-1-su6vt_1  | Requirement already satisfied: numpy>=1.13.3 in /usr/local/lib/python3.6/site-packages (from pandas==1.0.0->-r /opt/ml/model/code/requirements.txt (line 1)) (1.17.4)
algo-1-su6vt_1  | Collecting pytz>=2017.2
algo-1-su6vt_1  |   Downloading pytz-2020.1-py2.py3-none-any.whl (510 kB)
     |████████████████████████████████| 510 kB 38.1 MB/s eta 0:00:01
algo-1-su6vt_

algo-1-su6vt_1  | 2020-08-31 13:50:30,212 [INFO ] pool-1-thread-6 ACCESS_LOG - /172.18.0.1:40730 "GET /ping HTTP/1.1" 200 38
!algo-1-su6vt_1  | 2020-08-31 13:50:31,888 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Generating new fontManager, this may take some time...
algo-1-su6vt_1  | 2020-08-31 13:50:31,906 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Generating new fontManager, this may take some time...
algo-1-su6vt_1  | 2020-08-31 13:50:31,927 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Generating new fontManager, this may take some time...
algo-1-su6vt_1  | 2020-08-31 13:50:31,942 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Generating new fontManager, this may take some time...
algo-1-su6vt_1  | 2020-08-31 13:50:33,030 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Using CPU
algo-1-su6vt_1  | 2020-08-31 13:50:33,045 [INFO ] W-9000-model-stdout com.amazo

로컬에서 컨테이너를 배포했기 때문에 컨테이너가 현재 실행 중임을 확인할 수 있습니다.

In [7]:
!docker ps

CONTAINER ID        IMAGE                                                                        COMMAND                  CREATED              STATUS              PORTS                              NAMES
62870f41de84        763104351884.dkr.ecr.us-east-1.amazonaws.com/mxnet-inference:1.6.0-cpu-py3   "python /usr/local/b…"   About a minute ago   Up About a minute   0.0.0.0:8080->8080/tcp, 8081/tcp   tmp65wkqgnh_algo-1-su6vt_1


### Inference using SageMaker SDK

SageMaker SDK의 `predict()` 메서드로 쉽게 추론을 수행할 수 있습니다. 

In [8]:
outputs = predictor.predict(input_data)

algo-1-su6vt_1  | 2020-08-31 13:51:49,344 [INFO ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 231
algo-1-su6vt_1  | 2020-08-31 13:51:49,345 [INFO ] W-9000-model ACCESS_LOG - /172.18.0.1:40792 "POST /invocations HTTP/1.1" 200 235


In [9]:
print(outputs)

{'item_0': [14.163518905639648, 16.821189880371094, 17.345623016357422, 18.332263946533203, 19.786230087280273, 20.85190200805664, 21.709440231323242, 14.616772651672363, 16.98798370361328, 17.16689109802246, 18.095056533813477, 19.661399841308594, 20.651142120361328, 21.328493118286133, 14.188490867614746, 16.60407066345215, 16.65607261657715, 17.947124481201172, 19.911487579345703, 20.073078155517578, 20.56958770751953], 'item_1': [36.81255340576172, 45.031005859375, 46.132381439208984, 48.175514221191406, 52.2011604309082, 55.74961471557617, 58.72719192504883, 39.19926834106445, 45.59088897705078, 45.85310363769531, 47.93359375, 52.64041519165039, 54.37112808227539, 58.041770935058594, 37.91728210449219, 43.920860290527344, 44.90949249267578, 47.97834014892578, 51.413307189941406, 54.659767150878906, 57.35679244995117]}


### Inference using Boto3 SDK

SageMaker SDK의 `predict()` 메서드로 추론을 수행할 수도 있지만, 이번에는 boto3의 `invoke_endpoint()` 메서드로 추론을 수행해 보겠습니다.<br>
Boto3는 서비스 레벨의 low-level SDK로, ML 실험에 초점을 맞춰 일부 기능들이 추상화된 high-level SDK인 SageMaker SDK와 달리
SageMaker API를 완벽하게 제어할 수 있습으며, 프로덕션 및 자동화 작업에 적합합니다.

참고로 `invoke_endpoint()` 호출을 위한 런타임 클라이언트 인스턴스 생성 시, 로컬 배포 모드에서는 `sagemaker.local.LocalSagemakerRuntimeClient()`를 호출해야 합니다.

In [10]:
client = sagemaker.local.LocalSagemakerClient()
runtime_client = sagemaker.local.LocalSagemakerRuntimeClient()
endpoint_name = local_model.endpoint_name

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='application/json',
    Accept='application/x-npy',
    Body=json.dumps(input_data)
    )
outputs = response['Body'].read().decode()

algo-1-su6vt_1  | 2020-08-31 13:51:51,739 [INFO ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 209
algo-1-su6vt_1  | 2020-08-31 13:51:51,740 [INFO ] W-9000-model ACCESS_LOG - /172.18.0.1:40796 "POST /invocations HTTP/1.1" 200 211


In [11]:
outputs

'{"item_0": [14.163518905639648, 16.821189880371094, 17.345623016357422, 18.332263946533203, 19.786230087280273, 20.85190200805664, 21.709440231323242, 14.616772651672363, 16.98798370361328, 17.16689109802246, 18.095056533813477, 19.661399841308594, 20.651142120361328, 21.328493118286133, 14.188490867614746, 16.60407066345215, 16.65607261657715, 17.947124481201172, 19.911487579345703, 20.073078155517578, 20.56958770751953], "item_1": [36.81255340576172, 45.031005859375, 46.132381439208984, 48.175514221191406, 52.2011604309082, 55.74961471557617, 58.72719192504883, 39.19926834106445, 45.59088897705078, 45.85310363769531, 47.93359375, 52.64041519165039, 54.37112808227539, 58.041770935058594, 37.91728210449219, 43.920860290527344, 44.90949249267578, 47.97834014892578, 51.413307189941406, 54.659767150878906, 57.35679244995117]}'

### Local Mode Endpoint Clean-up

엔드포인트를 계속 사용하지 않는다면, 엔드포인트를 삭제해야 합니다. 
SageMaker SDK에서는 `delete_endpoint()` 메소드로 간단히 삭제할 수 있습니다.

In [12]:
def delete_endpoint(client, endpoint_name):
    response = client.describe_endpoint_config(EndpointConfigName=endpoint_name)
    model_name = response['ProductionVariants'][0]['ModelName']

    client.delete_model(ModelName=model_name)    
    client.delete_endpoint(EndpointName=endpoint_name)
    client.delete_endpoint_config(EndpointConfigName=endpoint_name)    
    
    print(f'--- Deleted model: {model_name}')
    print(f'--- Deleted endpoint: {endpoint_name}')
    print(f'--- Deleted endpoint_config: {endpoint_name}')    
delete_endpoint(client, endpoint_name)

Gracefully stopping... (press Ctrl+C again to force)
--- Deleted model: mxnet-inference-2020-08-31-13-50-13-777
--- Deleted endpoint: local-endpoint-store-item-demand-forecast-1598881809
--- Deleted endpoint_config: local-endpoint-store-item-demand-forecast-1598881809


<br>

## 4. SageMaker Hosted Endpoint Inference
---

이제 실제 운영 환경에 엔드포인트 배포를 수행해 보겠습니다. 로컬 모드 엔드포인트와 대부분의 코드가 동일하며, 모델 아티팩트 경로(`model_data`)와 인스턴스 유형(`instance_type`)만 변경해 주시면 됩니다. SageMaker가 관리하는 배포 클러스터를 프로비저닝하는 시간이 소요되기 때문에 추론 서비스를 시작하는 데에는 약 5~10분 정도 소요됩니다.

In [13]:
import os
import boto3
import sagemaker
from sagemaker.mxnet import MXNet

boto_session = boto3.Session()
sagemaker_session = sagemaker.Session(boto_session=boto_session)
role = sagemaker.get_execution_role()
bucket = sagemaker.Session().default_bucket()

In [14]:
model_path = os.path.join(s3_model_dir, "model.tar.gz")
endpoint_name = "endpoint-store-item-demand-forecast-{}".format(int(time.time()))

In [15]:
model = MXNetModel(model_data=model_path,
                         role=role,
                         source_dir='src',
                         entry_point='inference.py',
                         framework_version='1.6.0',
                         py_version='py3')

predictor = model.deploy(instance_type="ml.c5.large", 
                         initial_instance_count=1, 
                         endpoint_name=endpoint_name,
                         wait=True)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


-------------!

추론을 수행합니다. 로컬 모드의 코드와 동일합니다.

In [16]:
outputs = predictor.predict(input_data)
print(outputs)

{'item_0': [14.163518905639648, 16.821189880371094, 17.345624923706055, 18.332263946533203, 19.786231994628906, 20.85190200805664, 21.70943832397461, 14.616772651672363, 16.987985610961914, 17.16689109802246, 18.09505844116211, 19.661399841308594, 20.651142120361328, 21.3284912109375, 14.188492774963379, 16.60407066345215, 16.656070709228516, 17.94712257385254, 19.911489486694336, 20.073078155517578, 20.56958770751953], 'item_1': [36.81254959106445, 45.031009674072266, 46.13237762451172, 48.175514221191406, 52.2011604309082, 55.74961471557617, 58.72719192504883, 39.19926452636719, 45.59089279174805, 45.85310363769531, 47.933597564697266, 52.64041519165039, 54.371124267578125, 58.041770935058594, 37.91728210449219, 43.920860290527344, 44.90949249267578, 47.97834014892578, 51.413299560546875, 54.65977478027344, 57.35679244995117]}


In [17]:
import boto3
client = boto3.client('sagemaker')
runtime_client = boto3.client('sagemaker-runtime')
endpoint_name = model.endpoint_name

In [18]:
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='application/json',
    Accept='application/json',
    Body=json.dumps(input_data)
    )
outputs = response['Body'].read().decode()

In [19]:
outputs

'{"item_0": [14.207303047180176, 16.919790267944336, 17.345754623413086, 18.348861694335938, 19.965818405151367, 20.748655319213867, 21.590065002441406, 14.480704307556152, 16.978944778442383, 16.658592224121094, 18.25176429748535, 19.607223510742188, 20.795198440551758, 21.353797912597656, 14.0289888381958, 16.48247718811035, 16.907812118530273, 17.71446990966797, 19.946365356445312, 20.2979736328125, 20.792892456054688], "item_1": [36.48078155517578, 44.30986404418945, 45.861942291259766, 48.04996109008789, 52.20718765258789, 56.05976104736328, 58.393402099609375, 38.8161735534668, 45.91603088378906, 45.5969352722168, 47.952693939208984, 52.254703521728516, 53.863525390625, 56.945255279541016, 37.448265075683594, 44.1944580078125, 44.1426887512207, 47.769264221191406, 51.138206481933594, 54.509674072265625, 57.06270980834961]}'

### SageMaker Hosted Endpoint Clean-up

엔드포인트를 계속 사용하지 않는다면, 불필요한 과금을 피하기 위해 엔드포인트를 삭제해야 합니다. 
SageMaker SDK에서는 `delete_endpoint()` 메소드로 간단히 삭제할 수 있으며, UI에서도 쉽게 삭제할 수 있습니다.

In [20]:
delete_endpoint(client, endpoint_name)

--- Deleted model: mxnet-inference-2020-08-31-13-52-41-698
--- Deleted endpoint: endpoint-store-item-demand-forecast-1598881931
--- Deleted endpoint_config: endpoint-store-item-demand-forecast-1598881931
